# RVC 1, Ch2.1

In [ ]:
# Works best with jupyter-notebook

In [3]:
%matplotlib notebook 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import numpy as np

from spatialmath import *
from spatialmath.base import *
#from roboticstoolbox import *
import matplotlib.pyplot as plt

#!pip3 install sympy testresources pgraph

np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

## Lec 01.6 (Ch 2.1) 2D rotations
*For more help with 2D commands see:*
https://petercorke.github.io/spatialmath-python/func_2d.html

Rotations about a given axis can be achieved with:
- SO(2) rot2 (ndarray (2,2)) rotation matrices
- SE(2) SE2 creates (ndarray (3,3)) rotation matrices

In [5]:
# Create a rotation matrix
R = rot2(0)
print(R)
type(R)

[[       1        0]
 [       0        1]]


numpy.ndarray

Rotate by 0.2 radians

In [6]:
R = rot2(0.2)
print(R)

[[  0.9801  -0.1987]
 [  0.1987   0.9801]]


Rotate by 30 degrees

In [7]:
R = rot2(30,'deg')
print(R)

[[   0.866     -0.5]
 [     0.5    0.866]]


Rotation matrices are 
 1. orthnormal: columns are orthogonal to each other and unit length
 2. full rank: independent/invertible
 3. symmetrical: the inverse is equal to the transpose

In [8]:
c1 = R[:,0]
c2 = R[:,1]

print('Norm c1: ', norm(c1))
print('Norm c2: ', np.linalg.norm(c2))

Norm c1:  1.0
Norm c2:  1.0


Dot Products:
    
There are several ways to compute the dot products. Directly from np.dot(a,b) or calling dot on a nupy array variable:

In [9]:
np.dot(c1,c2) # np.dot() works with vectors, dot() works with Quaternions
c1.dot(c2)

0.0

Determinant:
    
Numpy also has methods to compute the determinant of a matrix. Non zero determinants indicate full rank.

In [10]:
np.linalg.det(R) # lnp.lingalg builds on blas/lapack libs. See https://numpy.org/doc/stable/reference/routines.linalg.html

1.0

Inverse:

Computing the inverse and the transpose is easy. Rotation matrices have a special property that their inverse is equal to their transpose. 

In [11]:
np.linalg.inv(R)

array([[   0.866,      0.5],
       [    -0.5,    0.866]])

In [15]:
# Symmetrical
R_transpose = np.transpose(R)
print(f'The original rotation matrix is: \n{R}.')
print(f'The transposed rotation matrix R\' is: \n{R_transpose}.')

The original rotation matrix is: 
[[   0.866      0.5]
 [    -0.5    0.866]].
The transposed rotation matrix R' is: 
[[   0.866     -0.5]
 [     0.5    0.866]].


Matrix-Matrix Multiplication:

Matrix multiplication can be done in multiple ways. 

Rotation matrices keep all of their properties even when we multiply them.

Be careful not to confuse it with element-wise matrix multiplication

In [16]:
# Matmul
print('Using np.matmul: \n', np.matmul(R, R) ,'\n')

# @ operator
print('Using @ operator: \n', R@R, '\n')

# Careful not to confuse with elementwise multiplication.
print('R*R is element-wise multiplication: \n', R*R)

Using np.matmul: 
 [[     0.5    0.866]
 [  -0.866      0.5]] 

Using @ operator: 
 [[     0.5    0.866]
 [  -0.866      0.5]] 

R*R is element-wise multiplication: 
 [[    0.75     0.25]
 [    0.25     0.75]]


## Symbolic

Pyton's symbolic package will be useful to work with mathematical expressions directly. To use it, make sure you have installed it first. 

Installation occurs from bash not inside the python interpreter. But you can call bash commands from inside the interpreter if you use the exclamation mark !

In [17]:
# Run as shell command with !
# !pip3 install sympy

In [18]:
from spatialmath.base.symbolic import *

In [19]:
theta = symbol('theta')

R = rot2(theta)
print(R)

[[cos(theta) -sin(theta)]
 [sin(theta) cos(theta)]]


In [20]:
R2 = np.matmul(R,R)
print(R2)

[[-sin(theta)**2 + cos(theta)**2 -2*sin(theta)*cos(theta)]
 [2*sin(theta)*cos(theta) -sin(theta)**2 + cos(theta)**2]]


Notice, that the symoblic class allows us to call the simplify method to get a much nicer representation that exploits identities. 

In [21]:
simplify(np.matmul(R,R))

[[cos(2*theta), -sin(2*theta)], [sin(2*theta), cos(2*theta)]]

---
## Lec 01.7 (Ch 2.1) 2D Rotation and Translation

Create a homogenous transformation representation of a pure translation working in 2D.


In [22]:
# https://petercorke.github.io/spatialmath-python/func_2d.html?highlight=transl2#spatialmath.base.transforms2d.transl2
T1 = transl2(1, 2)

print(type(T1))
print(T1)

<class 'numpy.ndarray'>
[[       1        0        1]
 [       0        1        2]
 [       0        0        1]]


Create a homogenous transformation representation a pure rotation

In [25]:
R1 = rot2(0.5236)
print(type(R))
print(f'2D Rotation matrix R1 about 0.5236 radians is: \n{R1}')

R1 = rot2(30,'deg')
print(f'2D Rotation matrix R1 about 30 degrees is: \n{R1}')

<class 'numpy.ndarray'>
2D Rotation matrix R1 about 0.5236 radians is: 
[[   0.866     -0.5]
 [     0.5    0.866]]
2D Rotation matrix R1 about 30 degrees is: 
[[   0.866     -0.5]
 [     0.5    0.866]]


Rotation + Translation:

We can easily generate a 2D homogenous tranformation matrix that includes a rotation and translation by multiplying the individual compoents. The @operator is particularly convenient:

In [28]:
T1 = transl2(1, 2)@trot2(30, 'deg')

print(type(T1))
print(f'The homogeneous transformation matrix is: \n{T1}.')

<class 'numpy.ndarray'>
The homogeneous transformation matrix is: 
[[   0.866     -0.5        1]
 [     0.5    0.866        2]
 [       0        0        1]].


Special Eucliden SE(2) command can combine the translation and rotation with a single command.

But notice that in this case, the output object is not of type numpy.ndarray, but SE2.

SE2 objects share same methods as lists... but if you want to extract elements, you need to convert it to a numpy.ndarray object first using the .A attribute.

In [32]:
T1 = SE2(1,2,30, unit='deg') # yields an SE object
print(type(T1))
print(f'The special SE2 object (homogeneous transform) is: \n{T1}')

print('The version instantiated by radians is: ')
T1 = SE2(1,2,0.5236)
print(T1)

<class 'spatialmath.pose2d.SE2'>
The special SE2 object (homogeneous transform) is: 
   0.866    -0.5       1         
   0.5       0.866     2         
   0         0         1         

The version instantiated by radians is: 
   0.866    -0.5       1         
   0.5       0.866     2         
   0         0         1         



Try to extract the [1,1] element of the T1 homogeneous transform. 

Extract the numpy array property first, and then use square brackets:

In [34]:
# Extract numpy.ndarray version first.
T1.A[1,1]

0.866024791582939

The following does not work. 

In [36]:
try:
    T1[1,1] # does not work as an SE2 object
except:
    print('Cannot access directly')

Cannot access directly


### Plotting

Plotting will be done via matplotlib, there are several other packages that you could use in python to plot (i.e. plotly).

You can explore different tutorials online on matplotlib to understand the package better. 

You will want to understand differences between fixures and axes and different ways to represent data and labels. Learning about backends is also important to modify the interfaces. 

https://matplotlib.org/3.1.1/tutorials/index.html
https://www.tutorialspoint.com/matplotlib/index.htm


In [ ]:
# Setup plotting capabilities for Jupyterlab
# Run as shell command with !
#!pip3 install ipympl

Getting ready to plot. 

In [ ]:
# Create a figure
fig = plt.figure() # Save the figure handle in object fig

In [ ]:
# Manipulate the axes of fig above
plt.axes( xlim = (-1,5),
          ylim = (-1,6) ); # Change the dimensions of selected axes

In [ ]:
# Create new coordinates
T1 = transl2(1, 2)

Plot the homogenous transform with trplot2. In fact, trplot 2 will create a figure by itself, we do not need to explicitly call it. If a figure is already open, trplot will use it.

In [ ]:
# https://petercorke.github.io/spatialmath-python/func_2d.html?highlight=trplot#spatialmath.base.transforms2d.trplot2
trplot2(T1, frame='1', color='blue', labels=('x','y'),block=False); # assign color and labels

Additional trplots, will use the same figure, unless one explicitly creates/selects a different figure.

In [ ]:
T2 = transl2(2, 1)
trplot2(T2, frame='2', color='red',block=False);

How would a compound transformation T1 followed by T2 look like

In [ ]:
T3 = T1@T2
trplot2(T3, frame='3', color='green', block=False);

In [ ]:
# Create a point W_P wrt to the world
P = np.array([[3],[2]])

Points can be represented via the 'scatter' plot in matplotlib. We can assign the color of the point via option 'c' as well as the type of marker via option 'marker'.

In [ ]:
plt.scatter(P[0].item(),P[1].item(), c='black',marker='^' );

What if we wish to know the coordinates of the point wrt to coordinate frame 1 and not the world origin?

In [ ]:
# Where, 1_T_W = inv(W_T_1)
# 1_P = 1_T_W * W_P
P1 = np.matmul(np.linalg.inv(T1),e2h(P))
print(P1)

In [ ]:
# Plot the point
plt.scatter( h2e(P1)[0],h2e(P1)[1],c='red');
print(h2e(P1))

In [ ]:
# 2_P = 2_T_W * W_P
P2 = np.matmul(np.linalg.inv(T2),e2h(P))
print(P2)